# Build a Model

Dan thinks a next great step: 
* basis of building a model.
* i.e. try to predict star rating (1-5) based off review text... in class different options for taking in review text and converting it to something we can reason about.
* For each review, create something that is a 1-hot encoding of that review. i.e. for all words used in corpus.

In [ ]:
# some possible useful functions courtesy of DL exercise chat: 

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
from sklearn.model_selection import train_test_splitfrom sklearn.linear_model import LogisticRegressionfrom sklearn import metrics


x_train, x_test, y_train, y_test = train_test_split(reviews, ratings)

lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr')





In [2]:
!pip3 install spacy

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 6.2 MB 27.5 MB/s            
     |████████████████████████████████| 125 kB 53.0 MB/s            
     |████████████████████████████████| 10.1 MB 45.1 MB/s            
     |████████████████████████████████| 660 kB 40.7 MB/s            
     |████████████████████████████████| 181 kB 43.3 MB/s            
     |████████████████████████████████| 42 kB 2.7 MB/s             
     |████████████████████████████████| 457 kB 41.9 MB/s            
     |████████████████████████████████| 9.9 MB 43.8 MB/s            
     |████████████████████████████████| 58 kB 10.1 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-scr

In [11]:
!python -m spacy download en_core_web_sm

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 12.0 MB 25.1 MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py2-none-any.whl size=12047107 sha256=83835ef63b7e2cf2f1cc5f17db4b15adb345c2bd8b7be255ec7a88d06345eb83
  Stored in directory: /home/ec2-user/.cache/pip/wheels/b5/bb/72/85633611a182045b6a28abebca619bb7ae8693c4e95fef1188
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the mo

In [3]:
import gzip
import json
import spacy
import pandas as pd

In [4]:
# Read in grocery and gourmet food data

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [5]:
df = getDF('Grocery_and_Gourmet_Food_5.json.gz')

In [6]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"11 19, 2014",A1QVBUH9E1V6I8,4639725183,Jamshed Mathur,No adverse comment.,Five Stars,1416355200,NaN,NaN,NaN
1,5.0,True,"10 13, 2016",A3GEOILWLK86XM,4639725183,itsjustme,Gift for college student.,Great product.,1476316800,NaN,NaN,NaN
2,5.0,True,"11 21, 2015",A32RD6L701BIGP,4639725183,Krystal Clifton,"If you like strong tea, this is for you. It mi...",Strong,1448064000,NaN,NaN,NaN
3,5.0,True,"08 12, 2015",A2UY1O1FBGKIE6,4639725183,U. Kane,Love the tea. The flavor is way better than th...,Great tea,1439337600,NaN,NaN,NaN
4,5.0,True,"05 28, 2015",A3QHVBQYDV7Z6U,4639725183,The Nana,I have searched everywhere until I browsed Ama...,This is the tea I remembered!,1432771200,NaN,NaN,NaN


In [7]:
# Interested in columns overall and reviewText

df_data = df[["overall", "reviewText"]]

In [8]:
# Clean reviewText

# import spacy 
print("Row count is:", len(df_data.index))
print("Number of reviewText null is: ", df_data["reviewText"].isnull().values.sum())
print("Number of overall null is: ", df_data["overall"].isnull().values.sum())

df_data = df_data.dropna()
print("Clean row count is:", len(df_data.index))


Row count is: 1143860
Number of rewiewText null is:  390
Number of overall null is:  0
Clean row count is: 1143470


In [43]:
# Use spacy to remove stop words and craete list of words for each reviewText row?
# nlp = spacy.load('en_core_web_sm')
from sklearn.feature_extraction import text
stop = text.ENGLISH_STOP_WORDS

# Should all these transforms be in my read function? Probably... so I can load data efficiently. 
# I think I'm thinking about this wrong.
# split string into list, all lower case, remove punctuation, remove stop words
df_data['text_split'] = df_data['reviewText'].str.replace('[^\w\s]','').str.lower().str.split()
df_data['text_no_stop'] = df_data['text_split'].apply(lambda x: [item for item in x if item not in stop])



/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:7: FutureWarning: The default value of regex will change from True to False in a future version.


In [44]:
df_data.head(5)

,overall,reviewText,text_split,text_no_stop
0,5.0,No adverse comment.,"[no, adverse, comment]","[adverse, comment]"
1,5.0,Gift for college student.,"[gift, for, college, student]","[gift, college, student]"
2,5.0,"If you like strong tea, this is for you. It mi...","[if, you, like, strong, tea, this, is, for, yo...","[like, strong, tea, little, strong]"
3,5.0,Love the tea. The flavor is way better than th...,"[love, the, tea, the, flavor, is, way, better,...","[love, tea, flavor, way, better, regular, lipt..."
4,5.0,I have searched everywhere until I browsed Ama...,"[i, have, searched, everywhere, until, i, brow...","[searched, browsed, amazon, tea, lipton, selli..."


In [ ]:
# remove punctuation and lemmanize words... 

In [21]:
df_data.to_csv("df_data_split.csv")  

In [19]:
def to_doc(words:tuple) -> spacy.tokens.Doc:
    # Create SpaCy documents by joining the words into a string
    return nlp(' '.join(words))

def remove_stops(doc) -> list:
    # Filter out stop words by using the `token.is_stop` attribute
    return [token.text for token in doc if not token.is_stop]

def lemmatize(doc) -> list:
    # Take the `token.lemma_` of each non-stop word
    return [token.lemma_ for token in doc if not token.is_stop]

In [27]:
print(stop)

frozenset({'had', 'inc', 'further', 'throughout', 'i', 'made', 'eg', 'should', 'until', 'its', 'sixty', 'thereby', 'are', 'around', 'who', 'ours', 'never', 'beside', 'everyone', 'hereafter', 'several', 'formerly', 'yet', 'thereupon', 'me', 'him', 'yours', 'everything', 'might', 'serious', 'so', 'anything', 'it', 'off', 'well', 'toward', 'ie', 'therefore', 'must', 'up', 'former', 'ever', 'whatever', 'most', 'etc', 'five', 'put', 'that', 'himself', 'whereupon', 'cannot', 'empty', 'show', 'afterwards', 'somewhere', 'neither', 'top', 'amongst', 'nor', 'thru', 'yourselves', 'her', 'part', 'beyond', 'four', 'together', 'seeming', 'as', 'of', 'whom', 'much', 'find', 'go', 'whenever', 'noone', 'fill', 'may', 'whereafter', 'someone', 'something', 'us', 'less', 'please', 'ltd', 'same', 'anywhere', 'another', 'thus', 'ten', 'within', 'among', 'not', 'onto', 'other', 'perhaps', 'except', 'hers', 'wherever', 'to', 'against', 'bottom', 'one', 'you', 'otherwise', 'thence', 'still', 'a', 'thin', 'sinc

In [24]:
df_data.head(5)

,overall,reviewText,text_split
0,5.0,No adverse comment.,"[No, adverse, comment.]"
1,5.0,Gift for college student.,"[Gift, for, college, student.]"
2,5.0,"If you like strong tea, this is for you. It mi...","[If, you, like, strong, tea,, this, is, for, y..."
3,5.0,Love the tea. The flavor is way better than th...,"[Love, the, tea., The, flavor, is, way, better..."
4,5.0,I have searched everywhere until I browsed Ama...,"[I, have, searched, everywhere, until, I, brow..."


In [39]:
# 

In [40]:
df_data.head(5)

,overall,reviewText,text_split,text_no_stop
0,5.0,No adverse comment.,"[no, adverse, comment.]","[adverse, comment.]"
1,5.0,Gift for college student.,"[gift, for, college, student.]","[gift, college, student.]"
2,5.0,"If you like strong tea, this is for you. It mi...","[if, you, like, strong, tea,, this, is, for, y...","[like, strong, tea,, you., little, strong, me.]"
3,5.0,Love the tea. The flavor is way better than th...,"[love, the, tea., the, flavor, is, way, better...","[love, tea., flavor, way, better, regular, lip..."
4,5.0,I have searched everywhere until I browsed Ama...,"[i, have, searched, everywhere, until, i, brow...","[searched, browsed, amazon, it!, tea, lipton, ..."


In [30]:
test_list = df_data['text_split'][3]
print(test_list)
new_list = [item for item in test_list if item not in stop]
print(new_list)

['Love', 'the', 'tea.', 'The', 'flavor', 'is', 'way', 'better', 'than', 'the', 'regular', 'Lipton', 'black', 'tea.', 'Definetly', 'worth', 'the', 'money.']
['Love', 'tea.', 'The', 'flavor', 'way', 'better', 'regular', 'Lipton', 'black', 'tea.', 'Definetly', 'worth', 'money.']


In [26]:
df_data.head()

,overall,reviewText,text_split
0,5.0,No adverse comment.,"[No, adverse, comment.]"
1,5.0,Gift for college student.,"[Gift, for, college, student.]"
2,5.0,"If you like strong tea, this is for you. It mi...","[If, you, like, strong, tea,, this, is, for, y..."
3,5.0,Love the tea. The flavor is way better than th...,"[Love, the, tea., The, flavor, is, way, better..."
4,5.0,I have searched everywhere until I browsed Ama...,"[I, have, searched, everywhere, until, I, brow..."


In [22]:
# create documents for all tuples of tokens
docs = list(map(to_doc, df_data.text_split))


KeyboardInterrupt: 

In [20]:
# create documents for all tuples of tokens
# V Slow. Hmm
docs = list(map(to_doc, df_data.text_split))

# apply removing stop words to all
df_data['removed_stops'] = list(map(remove_stops, docs))

# apply lemmatization to all
df_data['lemmatized'] = list(map(lemmatize, docs))

KeyboardInterrupt: 

In [ ]:
df_data.head(5)

In [ ]:
df_data.to_csv("df_data_clean.csv")  

In [ ]:
# create corpus and corresponding one-hot dictionary
# something like:
all_words = ' '.join([word for word in df['lemmatized']])
corpus = set(all_words)
word_to_ix = {word: i for i, word in enumerate(corpus)}

In [ ]:
# Do I need to make everything lowercase or does lemmatize take care of that? (.str().lower())